In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import h5py
from scipy.cluster.hierarchy import fcluster, linkage
from collections import Counter
import pickle

import os
import pandas as pd
import numpy as np
import h5py
from collections import Counter
import pickle
import pandas as pd


with open('../../processed_data/selected_phenotypes.txt', 'rb') as reader:
    id2ph = [i.strip() for i in reader.readlines()]
    
with open('../src/ensembl2genename.p', 'rb') as reader:
    gid2gn = pickle.load(reader)

In [2]:
try:
    import cPickle as pickle
except:
    import pickle

In [3]:
path_to_processed_data = '../../processed_data/'
path_to_wgcna = '../../R_script/WGCNA/'
path_to_save = '../../gwas_component/eigengene_model/'

In [4]:
#!/usr/bin/env python
import pandas
import sqlite3
from scipy.stats import norm 
import statsmodels.stats.multitest as multi

id2tissue = os.listdir('../../R_script/WGCNA/')

for tissue0 in id2tissue:
    if tissue0 == 'Cells_EBV_transformed_lymphocytes':
        tissue = 'Cells_EBV-transformed_lymphocytes'
    else:
        tissue = tissue0

    df_1k = pd.read_csv(path_to_processed_data + 'predixcan_1kgenome/TW_%s_0.5.db_predicted_expression.txt.gz' % tissue, sep = '\t')
    df_1k.columns = [g.split('.')[0] for g in df_1k.columns]

    dfm = pd.read_csv(path_to_wgcna + '%s/gene_module.csv' % tissue0, header = None)

    with open(path_to_processed_data + 'model_genes/var/%s-gene2var.p' % tissue0, 'rb') as reader:
        gene2var = pickle.load(reader)
    gene2var = {g.split('.')[0]:v for g, v in gene2var.items()}

    with open(path_to_processed_data + 'model_genes/%s-model_genes.txt' % tissue0, 'rb') as reader:
        id2gene = [s.strip() for s in reader.readlines()]
    id2gene = np.array(id2gene)

    df_exp = pd.read_csv(path_to_processed_data + 'adj_expression/%s-adj_expression.txt.gz' % tissue0, index_col=0)
    df_exp.columns = [g.split('.')[0] for g in df_exp.columns]

    df_exp = df_exp[id2gene] 
    data = df_exp.values

    clf = PCA(n_components = 1, whiten=False)

    eigen_genes = []
    res_ratio = []
    ## PCA
    ind_c = dfm[1].values

    pca_models = []
    
    for c in set(ind_c):
        if np.sum(ind_c == c) < 2:
            continue

        data0 = data[:, ind_c == c]
        clf.fit(data0)

        tmp = clf.transform(data0)

        ## gathering model weights
        
        pca_weight = np.squeeze(clf.components_)
        pca_genes = id2gene[ind_c == c]
        
        sig_1k = (df_1k[pca_genes].values * pca_weight[np.newaxis, :]).sum(axis = 1).std()
        
        pca_models.append([c, pca_genes, pca_weight, sig_1k])

        eigen_genes.append(tmp)
        res_ratio.append(clf.explained_variance_ratio_[0])
    
    eigen_genes = np.hstack(eigen_genes)      

    with open(path_to_save + '%s.p' % tissue0, 'wb') as writer:
        pickle.dump(pca_models, writer)
